<a href="https://colab.research.google.com/github/kgreed4/AIPI590_CV_Proj1/blob/kgreed/test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import os
import torch
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, precision_recall_curve
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models

In [2]:
filepath = '/content/drive/MyDrive/AIPI 590: Computer Vision/Project01'

In [37]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

class CustomDataset(Dataset):
    def __init__(self, csv_file, base_dir=filepath, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            img_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
            base_dir (string): path of base directory.
        """
        self.data_frame = pd.read_csv(os.path.join(filepath,csv_file))
        self.base_dir = filepath
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        img_name = os.path.join(self.base_dir, self.data_frame.iloc[idx, 0])

        image = Image.open(img_name).convert('RGB')

        label = self.data_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)

        # Map class names to numerical labels
        self.class_to_label = {
            'dew': 0,
            'fogsmog': 1,
            'frost': 2,
            'glaze': 3,
            'hail': 4,
            'lightning': 5,
            'rain': 6,
            'rainbow': 7,
            'rime': 8,
            'sandstorm': 9,
            'snow': 10
        }
        label = self.class_to_label[label]
        return image, label

In [31]:
def load_datasets(train_csv, val_csv, test_csv, base_dir=filepath, augment=False, balance_classes=False):
    # Apply any transformations here
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    # augment training data
    if augment:
        csv_output = os.path.join(filepath, 'data-csvs/augmented_dataset.csv')
        augment_data(train_csv, csv_output, output_folder='augmented_dataset')
        train_csv = csv_output

    # balance the classes
    if balance_classes:
        csv_output = os.path.join(filepath, 'data-csvs/balanced_augmented_dataset.csv')
        balance_dataset(train_csv, csv_output, output_folder='balanced_augmented_dataset')
        train_csv = csv_output

    train_dataset = CustomDataset(csv_file=train_csv, base_dir=base_dir, transform=transform)
    val_dataset = CustomDataset(csv_file=val_csv, base_dir=base_dir, transform=transform)
    test_dataset = CustomDataset(csv_file=test_csv, base_dir=base_dir, transform=transform)

    return train_dataset, val_dataset, test_dataset

def setup_dataloaders(train_csv=filepath+'/data-csvs/train_images_labeled.csv', val_csv=filepath+'/data-csvs/valid_images_labeled.csv', test_csv=filepath+'/data-csvs/test_images_labeled.csv', batch_size=32, augment=False, balance_classes=False):
    train_dataset, val_dataset, test_dataset = load_datasets(train_csv, val_csv, test_csv, augment=augment, balance_classes=balance_classes)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

In [39]:
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.preprocessing import label_binarize

"""
    Function to run the model on the test data, calculate evaluation metrics,
    and visualize performance.

    Parameters:
        model: PyTorch model to be tested.
        test_dl: DataLoader containing test data.
        pathtocsv: Path to the CSV file containing test data labels.
        nameOfModel: Name of the model to be used in file name for test results.
    """
def test_model(model_path, test_loader, model_name):

    # Put the model in evaluation mode
    model.eval()

    # Initialize lists to store true labels and predicted labels
    true_labels = []
    pred_labels = []
    pred_probs = []

    # Unmap numerical labels to original class names
    class_to_label = {
        0: 'dew',
        1: 'fogsmog',
        2: 'frost',
        3: 'glaze',
        4: 'hail',
        5: 'lightning',
        6: 'rain',
        7: 'rainbow',
        8: 'rime',
        9: 'sandstorm',
        10: 'snow'
    }

    # Iterate through the test data and make predictions
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        with torch.no_grad():
            outputs = model(images)
            _, predicted = torch.max(outputs, 1)
            softmax_probs = torch.nn.functional.softmax(outputs, dim=1)

        true_labels.extend(labels.cpu().numpy())
        pred_labels.extend(predicted.cpu().numpy())
        pred_probs.extend(softmax_probs.cpu().numpy())

    print('pred labels 1', pred_labels)

    # Unmap numerical labels to original class names
    true_labels = [class_to_label[label] for label in true_labels]
    print(true_labels)
    for label in pred_labels:
      print('preds class', class_to_label)
      print('preds label', class_to_label[label])
    pred_labels = [class_to_label[label] for label in pred_labels]

    # Convert true and predicted labels to one-hot encoding
    true_labels_onehot = label_binarize(true_labels, classes=class_to_label.values())
    pred_probs = np.array(pred_probs)

    # Compute overall accuracy, precision, recall, and F1-score
    accuracy = accuracy_score(true_labels, pred_labels)
    precision = precision_score(true_labels, pred_labels, average='weighted')
    recall = recall_score(true_labels, pred_labels, average='weighted')
    f1 = f1_score(true_labels, pred_labels, average='weighted')

    # Write evaluation metrics to a text file
    evaluation_file = f"{model_name}_evaluation.txt"
    with open(evaluation_file, 'w') as f:
        f.write(f"Model: {model_name}\n")
        f.write(f"Accuracy: {accuracy:.4f}\n")
        f.write(f"Precision: {precision:.4f}\n")
        f.write(f"Recall: {recall:.4f}\n")
        f.write(f"F1-score: {f1:.4f}\n\n")

    # Print evaluation metrics
    print(f"Model: {model_name}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1-score: {f1:.4f}")

    # Compute Precision-Recall curve and ROC curve per class
    precision = dict()
    recall = dict()
    fpr = dict()
    tpr = dict()
    roc_auc = dict()
    pr_auc = dict()

    for i, class_name in enumerate(class_to_label.values()):
        class_index = list(class_to_label.values()).index(class_name)
        precision[class_name], recall[class_name], _ = precision_recall_curve(true_labels_onehot[:, class_index], pred_probs[:, class_index])
        fpr[class_name], tpr[class_name], _ = roc_curve(true_labels_onehot[:, class_index], pred_probs[:, class_index])
        roc_auc[class_name] = auc(fpr[class_name], tpr[class_name])
        pr_auc[class_name] = auc(recall[class_name], precision[class_name])

    # Plot Precision-Recall curves
    plt.figure(figsize=(10, 8))
    plt.title('Precision-Recall Curve per Class')
    for class_name in class_to_label.values():
        plt.plot(recall[class_name], precision[class_name], label=f'{class_name} (AUC = {pr_auc[class_name]:.2f})')
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig(f"{model_name}_precision_recall_curve.png")
    plt.show()

    # Plot ROC curves
    plt.figure(figsize=(10, 8))
    plt.title('ROC Curve per Class')
    for class_name in class_to_label.values():
        plt.plot(fpr[class_name], tpr[class_name], label=f'{class_name} (AUC = {roc_auc[class_name]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend(loc='best')
    plt.grid(True)
    plt.savefig(f"{model_name}_roc_curve.png")
    plt.show()


In [40]:
# Load the model, specifying map_location to load it on GPU
state_dict = torch.load(os.path.join(filepath, 'resNet18_2.pth'), map_location=torch.device('cuda'))

# Instance
model = models.resnet18(pretrained=True)
model.load_state_dict(state_dict, strict=False)

# set up data loaders
train_loader, val_loader, test_loader = setup_dataloaders(balance_classes=False, augment=False)
dataloaders = {
    'train': train_loader,
    'val': val_loader,
    'test': test_loader
}

# Move the model to GPU if available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)
print(f"Model moved to {device}")

# Move the test data to the GPU
test_data = []
for data, target in test_loader:
    data, target = data.to(device), target.to(device)
    test_data.append((data, target))

test_model(model, test_data, 'resNet18_2')

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Model moved to cuda:0
pred labels 1 [644, 955, 772, 778, 59, 132, 31, 695, 791, 953, 879, 395, 62, 948, 675, 147, 93, 793, 107, 73, 693, 991, 29, 953, 386, 760, 718, 251, 541, 236, 749, 455, 103, 618, 989, 973, 561, 953, 642, 554, 997, 113, 420, 31, 601, 19, 995, 459, 488, 656, 428, 112, 705, 956, 32, 828, 598, 327, 348, 408, 974, 861, 202, 677, 721, 450, 879, 235, 395, 501, 539, 728, 306, 469, 300, 961, 937, 492, 858, 207, 489, 920, 840, 937, 926, 31, 989, 904, 644, 210, 633, 772, 524, 30, 347, 633, 436, 862, 612, 953, 23, 956, 790, 196, 818, 691, 738, 699, 995, 299, 937, 534, 133, 567, 864, 989, 24, 396, 746, 313, 758, 842, 574, 354, 332, 207, 563, 522, 228, 520, 55, 626, 50, 634, 127, 451, 257, 907, 279, 327, 996, 55, 489, 990, 879, 926, 772, 150, 568, 109, 181, 950, 579, 279, 115, 756, 657, 885, 956, 955, 150, 31, 518, 334, 266, 909, 488, 489, 354, 870, 978, 964, 979, 746, 994, 455, 468, 539, 309, 654, 812, 377, 199, 991, 973, 329, 470, 435, 977, 606, 505, 348, 468, 313, 795, 442, 

KeyError: 644

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)